# TP – Analyse de la qualité de l'air
## Etape 1 – Exploration et chargement Spark

### Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

### Créer une session Spark locale:

In [2]:
try:
    spark = SparkSession.builder \
        .appName("TP - Analyse de la qualité de l'air - Etape 1: Exploration et chargement Spark") \
        .master("local[*]") \
        .getOrCreate()
    print(f"\n[ok]: Spark local session creation successful.\n")
except Exception as e:
    print(f"\n[ko]: Spark local session creation failed: {e}\n")
    sys.exit(1)

spark.sparkContext.setLogLevel("ERROR") # keep only error+


[ok]: Spark local session creation successful.



### Charger `air_quality_raw.csv` en DataFrame Spark:

In [3]:
data_file_air_quality = "../data/air_quality_raw.csv"
try:
    df_air = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .option("sep", ",") \
            .csv(data_file_air_quality)
    print(f"\n[ok]: read source data file successfull: '{data_file_air_quality}'\n")
except Exception as e:
    print(f"\n[ko]: read source data file failed: {e}\n")
    sys.exit(1)


[ok]: read source data file successfull: '../data/air_quality_raw.csv'



### Afficher le schéma inféré et identifier les problèmes de typage:

In [4]:
df_air.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- pollutant: string (nullable = true)
 |-- value: string (nullable = true)
 |-- unit: string (nullable = true)



Problèmes de typage:
- timestamp: est en string => doit être en timestamp (TimestampType)
- value: est en string => doit être en double (DoubleType)

### Calculer des statistiques descriptives par polluant:

In [5]:
## Statistiques descriptives
df_air.groupBy("pollutant").agg(
    F.count("value").alias("count"),
    F.mean("value").alias("mean"),
    F.stddev("value").alias("stddev"),
    F.min("value").alias("min"),
    F.max("value").alias("max")
).show()

+---------+------+------------------+------------------+---+----+
|pollutant| count|              mean|            stddev|min| max|
+---------+------+------------------+------------------+---+----+
|       CO|205083|33.244708332371886|338.54961479208913|---|null|
|      NO2|205106| 78.76389141237787|340.06822809094757|---|null|
|       O3|205242|108.20850580284741|334.09565540179966|---|null|
|     PM10|205130| 70.21267468823184|335.66700132196564|---|null|
|    PM2.5|205215| 55.09891935232661| 335.5514556836554|---|null|
|      SO2|205175|  41.0071457848804|344.27247159941123|---|null|
+---------+------+------------------+------------------+---+----+



### Compter les valeurs nulles par colonne:

In [6]:
df_air.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_air.columns
]).show()

+----------+---------+---------+-----+----+
|station_id|timestamp|pollutant|value|unit|
+----------+---------+---------+-----+----+
|         0|        0|        0|    0|   0|
+----------+---------+---------+-----+----+



### Identifier les stations avec le plus d'enregistrements:

In [7]:
gp_station = df_air.groupBy("station_id").count().orderBy(F.desc("count")).show()


+----------+-----+
|station_id|count|
+----------+-----+
|    ST0032|26264|
|    ST0012|26244|
|    ST0028|26241|
|    ST0003|26239|
|    ST0029|26235|
|    ST0024|26235|
|    ST0020|26235|
|    ST0037|26233|
|    ST0023|26233|
|    ST0042|26224|
|    ST0015|26221|
|    ST0035|26221|
|    ST0007|26219|
|    ST0044|26207|
|    ST0030|26204|
|    ST0017|26199|
|    ST0014|26199|
|    ST0010|26199|
|    ST0009|26198|
|    ST0040|26198|
+----------+-----+
only showing top 20 rows



Les stations top 3 des stations avec le plus d'enregistrements: 
- La station **ST0032** avec 26264 enregistrement;
- La station **ST0012** avec 26244 enregistrement;
- La station **ST0028** avec 26241 enregistrement.

*La station qui a le moins d'enregistrements est la ST0040 avec 26198 enregistrement.*

### Synthèse des problèmes de qualité identifiés:
Problèmes de typage :
- timestamp: est en string => doit être en timestamp (TimestampType) ==> pas de forme unique valide en spark
- value: est en string => doit être en double (DoubleType) ==> pas de forme unique valide en spark (parfois des ,)

Valeurs négatives dans la colonne value (des polluants), ce qui est pas logique ;
Valeurs nulls également dans la colonne value (des polluants).